### Database Handler

In [181]:
import MySQLdb # database connector for MySQL
import time
import yaml
import pandas as pd

In [206]:
def get_exchange_id(exchange):
    """Get id of exchange"""
    status = cursor.execute("""select id from `exchange` where name = lower('{}')""".format(exchange))
    return(0 if status == 0 else cursor.fetchall()[0][0])

def get_currency_id(currency):
    """Get id of currency"""
    status = cursor.execute("""select id from currency where symbol = lower('{}')""".format(currency))
    return(0 if status == 0 else cursor.fetchall()[0][0])

def get_currency_pair_id(currency_pair):
    """Get id of currency pair"""
    status = cursor.execute("""select id from currency_pair where base_id = '{}' and counter_id = '{}'"""
                            .format(get_currency_id(currency_pair[0]),
                                    get_currency_id(currency_pair[1])))
    return(0 if status == 0 else cursor.fetchall()[0][0])

def get_strategy_id(name, code_file):
    """Get id of strategy"""
    status = cursor.execute("""select id from strategy where name = '{}' and code_file = '{}'""".format(name, code_file))
    return(0 if status == 0 else cursor.fetchall()[0][0])

def get_agent_id(strategy_id, param_file):
    """Get id of agent"""
    status = cursor.execute("""select id from agent where strategy_id = {} and param_file = '{}'"""
                            .format(strategy_id, param_file))
    return(0 if status == 0 else cursor.fetchall()[0][0])

def get_run_id(exchange_id, currency_pair_id, agent_id):
    """Get id of run"""
    status = cursor.execute("""select id from run where exchange_id = {} 
                            and currency_pair_id = {}
                            and agent_id = {}""".format(exchange_id, currency_pair_id, agent_id))
    return(0 if status == 0 else cursor.fetchall()[0][0])

def add_exchanges(exchanges):
    """Add a list of exchanges and return a list of ids"""
    ids = []
    for exchange in exchanges:
        cursor.execute("""insert ignore into `exchange` (name) values ('{}')""".format(exchange))
        ids.append(get_exchange_id(exchange))
        db.commit()
    return(ids)
    
def add_currencies(currencies):
    """Add a list of currencies and return a list of ids"""
    ids = []
    for currency in currencies:
        cursor.execute("""insert ignore into currency (symbol) values ('{}')""".format(currency.lower()))
        ids.append(get_currency_id(currency.lower()))
        db.commit()
    return(ids)

def add_currency_pairs(currency_pairs):
    """Add a list of currency pair tuples (base, counter) and return a list of ids"""
    ids = []
    for currency_pair in currency_pairs:
        base_id = get_currency_id(currency_pair[0])
        counter_id = get_currency_id(currency_pair[1])
        if base_id == 0 or counter_id == 0:
            print("Base or counter symbol missing in database. Add both currencies first before adding currency pair.")
            ids.append(0)
        else: 
            cursor.execute("""insert ignore into currency_pair (base_id, counter_id) values ({}, {})"""
                       .format(base_id, counter_id))
            db.commit()
            ids.append(get_currency_pair_id(currency_pair))
    return(ids)

def add_strategy(name, code_file):
    """Add a strategy and return its id"""
    cursor.execute("""insert ignore into strategy (name, code_file) values ('{}', '{}')""".format(name, code_file))
    db.commit()
    return(get_strategy_id(name, code_file))

def add_agent(strategy_id, param_file):
    """Add an agent tuples and return its id"""
    cursor.execute("""insert ignore into agent (strategy_id, param_file) values ({}, '{}')"""
                   .format(strategy_id, param_file))
    db.commit()
    return(get_agent_id(strategy_id, param_file))

def add_run(exchange_id, currency_pair_id, agent_id):
    """Add a run and return its id"""
    cursor.execute("""insert ignore into run (exchange_id, currency_pair_id, agent_id) values ({}, {}, {})"""
                  .format(exchange_id, currency_pair_id, agent_id))
    db.commit()
    return(get_run_id(exchange_id, currency_pair_id, agent_id))

def add_order(exchange_id, currency_pair_id, order_number, order_create_time,
                  run_id, type, price, order_amount, filled_amount, fee, status):
    """Add an order into database"""
    status = cursor.execute("""insert into order_summary 
                (exchange_id, currency_pair_id, order_number, order_create_time, 
                 run_id, type, price, order_amount, filled_amount, fee, status) 
                 values ({}, {}, '{}', '{}', {}, {}, {}, {}, {}, {}, {})"""
                .format(exchange_id, currency_pair_id, order_number, order_create_time,
                 run_id, type, price, order_amount, filled_amount, fee, status))
    db.commit()
    return(0 if status == 0 else cursor.lastrowid)

def get_all_orders():
    """Get all the orders and return as [[column names], tuple of tuples]"""
    status = cursor.execute("""select order_summary.id, timestamp, 
        order_create_time, order_number, 
        type, price, order_amount, filled_amount, fee, status,
        exchange.name as exchange, agent.param_file,
        strategy.name as strategy, strategy.code_file from order_summary 
        left join run on run.id = order_summary.run_id
        left join exchange on exchange.id = run.exchange_id
        left join agent on agent.id = run.agent_id
        left join strategy on strategy.id = agent.strategy_id""")
    column_names = [i[0] for i in cursor.description]
    return(0 if status == 0 else [column_names, cursor.fetchall()])

def get_all_runs():
    """Get all the runs and return as [[column names], tuple of tuples]"""
    status = cursor.execute("""select run.id, exchange.name as exchange, agent.param_file,
        strategy.name as strategy, strategy.code_file from run 
        left join exchange on exchange.id = run.exchange_id
        left join agent on agent.id = run.agent_id
        left join strategy on strategy.id = agent.strategy_id""")
    column_names = [i[0] for i in cursor.description]
    return(0 if status == 0 else [column_names, cursor.fetchall()])

def res_as_df(res):
    """Convert tuples from db fetches as panda dataframes"""
    return(pd.DataFrame(list(res[1]), columns = res[0]))

### Setting Up

Read from configuration file

In [138]:
with open("config.yml", 'r') as f:
    config = yaml.safe_load(f)
db = MySQLdb.connect(passwd=config['mysql']['passwd'], db=config['mysql']['db'], user=config['mysql']['user'])
cursor = db.cursor()

Adding exchanges, currency and currency pairs to support:

In [139]:
exchange_ids = add_exchanges(["bitfinex", "binance", "bitz", "huobipro"])
currency_ids = add_currencies(["btc", "eth", "usd"])
currency_pairs_ids = add_currency_pairs([("eth","btc"), ("btc","usd")])